# Investment Comp Screener

In [1]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm
import os
import glob
import re
import seaborn as sns

from statsmodels.distributions.empirical_distribution import ECDF

In [2]:
def calc_ecdf(data):
    #low is good
    ecdf= ECDF(data)
    current_perc = ecdf(data[-1])
    
    return(current_perc)

In [17]:
def color_negative_red(value):
    """
      Colors elements in a dateframe
      green if positive and red if
      negative. Does not color NaN
      values.
      """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

## Data Import

In [7]:
path = "holding_companies_data"
arr = [x for x in os.listdir(path) if x.endswith(".csv")]

In [8]:
for f in arr:
    print(f)

2020-09-04_cred a_pris_historik.csv
2020-09-04_inve b_pris_historik.csv
2020-09-04_kinv b_pris_historik.csv
2020-09-04_naxs_pris_historik.csv
2020-09-04_lato b_pris_historik.csv
2020-09-04_trac b_pris_historik.csv
2020-09-04_lund b_pris_historik.csv
2020-09-04_hav b_pris_historik.csv
2020-09-04_ores_pris_historik.csv
2020-09-04_indu c_pris_historik.csv
2020-09-04_svol b_pris_historik.csv
2020-09-04_vnv_pris_historik.csv
2020-09-04_bure_pris_historik.csv


In [139]:
for k in arr:
    if "trac" in k:
        traction = pd.read_csv(os.path.join(path,k))
    if "lund" in k:
        lundbergs = pd.read_csv(os.path.join(path,k))
    if "hav" in k:
        havsfrun = pd.read_csv(os.path.join(path,k))
    if "ores" in k:
        oresund = pd.read_csv(os.path.join(path,k))
    if "indu" in k:
        industri = pd.read_csv(os.path.join(path,k))
    if "svol" in k:
        svolder = pd.read_csv(os.path.join(path,k))
    if "bure" in k:
        bure = pd.read_csv(os.path.join(path,k))
    if "cred" in k:
        creades = pd.read_csv(os.path.join(path,k))
    if "inve" in k:
        investor = pd.read_csv(os.path.join(path,k))
    if "kinv" in k:
        kinnevik = pd.read_csv(os.path.join(path,k))
    if "vnv" in k:
        vnv = pd.read_csv(os.path.join(path,k))
    if "naxs" in k:
        naxs = pd.read_csv(os.path.join(path,k))
    if "lato" in k:
        latour = pd.read_csv(os.path.join(path,k))
    

## Calculate Metrics

In [140]:
look_back = 3*252


companies = [traction, lundbergs, havsfrun, oresund, industri,
             svolder, bure, creades, investor, kinnevik, vnv, naxs,latour]

companies_names = ['Traction', 'Lundbergsföretagen', 'Havsfrun Investment', 'Öresund', 'Industrivärden',
             'Svolder', 'Bure', 'Creades', 'Investor', 'Kinnevik', 'VNV Global', 'NAXS','Latour']

for comp in companies:
    
    comp['DATUM'] = pd.to_datetime(comp['DATUM'])
    comp.set_index(['DATUM'], inplace=True, drop=True)
    comp['R/P']=comp['PRIS']/comp['BERÄKNAT_SUBSTANSVÄRDE'] - 1
    



In [141]:
#high DSSP and HIDI is good

AVG_DISC = pd.DataFrame(latour['PRIS'].copy())
AVG_DISC['PRIS'] = 0
AVG_DISC.columns = ['AVG_DISC']

for comp in companies:

    AVG_DISC['AVG_DISC'] = AVG_DISC['AVG_DISC'] + comp['R/P']

AVG_DISC = AVG_DISC/len(companies)

for comp in companies:
    comp['SPREAD'] = comp['R/P'] - AVG_DISC['AVG_DISC']
    comp['DSSP'] = -comp['SPREAD'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['HIDI'] = -comp['R/P'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['MOM'] = (comp['PRIS'].pct_change(252) + comp['PRIS'].pct_change(126) + comp['PRIS'].pct_change(63))/3
    comp['NAV MOM'] = (comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(252) + comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(126) + comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(63))/3
    
    

In [142]:
INV_COMP = None

for comp in companies:
    
    INV_COMP = pd.concat([INV_COMP, comp.tail(1)], axis = 0)

INV_COMP['Company'] = companies_names
INV_COMP.set_index(['Company'], inplace=True, drop=True)

INV_COMP['Company'] = companies_names


In [143]:
INV_COMP['VRANK'] = (INV_COMP['R/P'].rank() + INV_COMP['DSSP'].rank(ascending = False) +
 INV_COMP['HIDI'].rank(ascending = False)).rank()



INV_COMP['MRANK'] = (INV_COMP['MOM'].rank(ascending = False)+INV_COMP['NAV MOM'].rank(ascending = False)).rank()

INV_COMP['COMB RANK'] = (INV_COMP['VRANK'] + INV_COMP['MRANK']).rank()


compact = ['R/P', 'DSSP', 'HIDI', 'MOM', 'NAV MOM', 'VRANK', 'MRANK', 'COMB RANK']

INV_COMP = INV_COMP[compact]

INV_COMP['R/P']=-INV_COMP['R/P']

In [145]:
average = pd.DataFrame(INV_COMP.mean(axis = 0)).transpose()
average.index = ['Average']
INV_COMP = pd.concat([INV_COMP, average], axis = 0)

## Display Results

In [146]:
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)

INV_COMP_VIEW = (INV_COMP.style
    .applymap(color_negative_red, subset=['R/P', 'MOM', 'NAV MOM'])
    .background_gradient(axis = 0 ,subset = ['DSSP', 'HIDI'],cmap=cm_high_good)
    .background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','COMB RANK'],cmap=cm_low_good)
    #.set_caption('Performance Metrics')
    .format({'R/P': "{:.1%}", 'DSSP': "{:.0%}", 'HIDI': "{:.0%}", 'MOM':"{:.1%}", 'NAV MOM': "{:.1%}",
            'Sortino': "{:.2f}", 'MAR': "{:.2f}"})
    .set_table_styles(styles))

In [147]:
INV_COMP_VIEW

,R/P,DSSP,HIDI,MOM,NAV MOM,VRANK,MRANK,COMB RANK
Traction,1.1%,97%,32%,-1.1%,-2.1%,3,11.5,9
Lundbergsföretagen,-6.0%,95%,28%,1.4%,6.9%,6,10,10
Havsfrun Investment,-37.9%,7%,5%,2.5%,-16.0%,10,11.5,13
Öresund,-4.2%,98%,43%,7.8%,4.1%,1,8.5,1
Industrivärden,9.1%,96%,19%,7.6%,8.4%,4,7,3.5
Svolder,2.3%,95%,17%,28.3%,22.0%,7,4,3.5
Bure,-24.2%,3%,2%,58.9%,27.3%,11.5,2,7
Creades,-64.5%,3%,3%,73.6%,29.8%,11.5,1,6
Investor,15.7%,92%,1%,15.7%,11.1%,8,6,8
Kinnevik,8.8%,79%,9%,45.4%,24.7%,9,3,5
